In [ ]:
import pandas as pd
import numpy as np

### Задание 1

Импортируйте файлы workouts, payments и users. Преобразуйте типы данных "дата+время"

In [ ]:
####

workouts = pd.read_csv('workouts.csv')
payments = pd.read_excel('payments.xlsx')
users = pd.read_excel('users.xlsx')

workouts['start_at'] = pd.to_datetime(workouts['start_at'])
users['first_contact_datetime'] = pd.to_datetime(users['first_contact_datetime'])

In [ ]:
workouts.head()

In [ ]:
payments.head()

In [ ]:
users.head()

### Задание 2

~~~ sql
select    workout_type
        , count(workout_id) as cnt
from workouts
group by workout_type
order by cnt desc
~~~

In [ ]:
####

workouts.groupby('workout_type').agg(cnt = ('workout_id', 'count')).reset_index().sort_values('cnt', ascending = False)

### Задание 3

~~~ sql
select    date_part('month', start_at) as dm
        , trainer_department
        , count(workout_id) as cnt
from workouts
group by   dm
         , trainer_department
order by dm
~~~

In [ ]:
####

workouts['dm'] = workouts['start_at'].dt.month
workouts.groupby(['dm', 'trainer_department']).agg(cnt = ('workout_id', 'count')).reset_index().sort_values('dm')

### Задание 4

~~~ sql
select    date_trunc('month', payment_date) as p_tm
        , sum(amount)/1000 as sum_amt
from payments
where workout_type = 'general'
group by p_tm
order by p_tm
~~~

In [ ]:
####

payments['p_tm'] = payments['payment_date'].dt.to_period('M').dt.to_timestamp()

result = payments[payments['workout_type'] == 'general'].groupby('p_tm').agg(sum_amt = ('amount', 'sum')).reset_index().sort_values('p_tm')
result['sum_amt'] = result['sum_amt'] /1000
round(result, 2)

### Задание 5

~~~ sql
select    user_id
        , min(date_trunc('day', payment_date)) as first_date
from payments p
    join users u
        on p.user_id = u.user_id
where region is not null
group by user_id
order by user_id
limit 20
~~~

In [ ]:
####

payments['first_date'] = payments['payment_date'].dt.to_period('D').dt.to_timestamp()
new_df = payments.merge(users, on = 'user_id', how = 'inner')
new_df[~new_df['region'].isnull()].groupby('user_id').agg({'first_date':'min'}).reset_index().sort_values('user_id').head(20)

### Задание 6

~~~ sql
select    avg(age) as avg_age
from users p
    left join payments u
        on p.user_id = u.user_id
where payment_date is null
~~~

In [ ]:
####

new_df1 = users.merge(payments, on = 'user_id', how = 'left')
new_df1[new_df1['payment_date'].isnull()].agg(avg_years_old = ('age', 'mean')).reset_index().rename(columns = {'index':'agregation_by'})

### Задание 7

~~~ sql
select   p_tm
       , sum(case when workout_type<>'general' then 1.0 else 0.0 end) / count(payment_id) as share_not_general
from payments
group by p_tm
~~~

In [ ]:
####

payments['p_tm'] = payments['payment_date'].dt.month
payments['nogeneral_flag'] = np.where(payments['workout_type'] != 'general', 1, 0)
df2 = payments.groupby('p_tm').agg({'nogeneral_flag':'sum', 'payment_id':'count'}).reset_index().rename(columns = {'payment_id':'cnt_pay_id'})
df2['share_not_general'] = df2['nogeneral_flag'] / df2['cnt_pay_id']
df2

### Задание 8

~~~ sql
select    w.clientId
        , cnt_workout
        , amt
from (select    client_id
              , count(workout_id) as cnt_workout
     from workouts
     group by client_id
     ) w
    join (select      user_id
                    , sum(amount) as amt
          from payments
          group by user_id
          ) p
        on p.user_id = w.client_id
limit 10
~~~

In [ ]:
####

selected_wkts = workouts.groupby('client_id').agg(cnt_workout = ('workout_id','count')).reset_index()
selected_paym = payments.groupby('user_id').agg(amt = ('amount','sum')).reset_index()
new_df3 = selected_wkts.merge(selected_paym, how = 'inner', left_on=['client_id'], right_on=
['user_id'])
new_df3 = new_df3.drop(columns = ['user_id'])
new_df3.head(10)

In [ ]:
################################################################################################################################
################################################################################################################################
################################################################################################################################